In [ ]:
#!pip install numpy requests nlpaug
#!pip install tensformers
import pandas as pd
import numpy as np
import nlpaug.augmenter.word as nlpaw
from tqdm import tqdm
from transformers import BertForSequenceClassification, AdamW, BertConfig,get_linear_schedule_with_warmup
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
# Note: this notebook requires torch >= 1.10.0
torch.__version__
device = "cuda" if torch.cuda.is_available() else "cpu"
device
from transformers import AutoTokenizer
from torch.utils.tensorboard import SummaryWriter
from transformers import DistilBertTokenizerFast
from transformers import AutoTokenizer
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil
# !pip install tensorboard
# !pip install accelerate -U
# !pip uninstall pillow
# !pip install pillow==9.4.0
# !pip install evaluate
from transformers import AutoModel, BertTokenizerFast
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:



df=pd.read_csv('../train-balanced-sarcasm.csv')
labels=df["label"].to_list()
comments=df["comment"].astype(str).to_list()
df2=pd.DataFrame({"comments":comments,"labels":labels})
df2_short={"text":comments,"labels":labels}

In [ ]:

x_train, x_test_valid, y_train, y_test_valid = train_test_split(comments, labels, test_size=0.33, random_state=42)

x_test, x_valid, y_test, y_valid = train_test_split(x_test_valid, y_test_valid, test_size=0.5, random_state=42)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(

    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id

)


# # import BERT-base pretrained model
# bert = AutoModel.from_pretrained('bert-base-uncased')

# # Load the BERT tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
     


In [ ]:
def preprocess_function(sent):
    
    return tokenizer (
                        sent,
                        # add_special_tokens = True,
                        # padding=True,
                        # max_length = 256,
                        truncation=True,
                        # return_attention_mask = True,
                        # return_tensors = 'pt',
                        
                   )

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels):
      self.encodings=encodings
      self.labels=labels   
    def __len__(self):
        return len(self.encodings)    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    #    return preprocess_function(self.data[idx]) ,torch.tensor(self.labels[idx])

train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_valid, truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)


train_dataset=CustomDataset(train_encodings,labels=y_train)
test_dataset=CustomDataset(val_encodings,labels=y_test)
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
test_dataloader=torch.utils.data.DataLoader(test_dataset)


In [ ]:


max_seq_len = 128
     


In [ ]:
# tokens_train = tokenizer.batch_encode_plus(
#     x_train,
#     max_length = max_seq_len,
#     pad_to_max_length=True,
#     truncation=True,
#     return_token_type_ids=False
# )

# # tokenize and encode sequences in the validation set
# tokens_val = tokenizer.batch_encode_plus(
#     x_valid,
#     max_length = max_seq_len,
#     pad_to_max_length=True,
#     truncation=True,
#     return_token_type_ids=False
# )

# # tokenize and encode sequences in the test set
# tokens_test = tokenizer.batch_encode_plus(
#     x_test,
#     max_length = max_seq_len,
#     pad_to_max_length=True,
#     truncation=True,
#     return_token_type_ids=False
# )
   

In [ ]:
# # for train set
# train_seq = torch.tensor(tokens_train['input_ids'])
# train_mask = torch.tensor(tokens_train['attention_mask'])
# train_y = torch.tensor(y_train)

# # for validation set
# val_seq = torch.tensor(tokens_val['input_ids'])
# val_mask = torch.tensor(tokens_val['attention_mask'])
# val_y = torch.tensor(y_valid)

# # for test set
# test_seq = torch.tensor(tokens_test['input_ids'])
# test_mask = torch.tensor(tokens_test['attention_mask'])
# test_y = torch.tensor(y_test)
     

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# #define a batch size
# batch_size = 32

# # wrap tensors
# train_data = TensorDataset(train_seq, train_mask, train_y)

# # sampler for sampling the data during training
# train_sampler = RandomSampler(train_data)

# # dataLoader for train set
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# # wrap tensors
# val_data = TensorDataset(val_seq, val_mask, val_y)

# # sampler for sampling the data during training
# val_sampler = SequentialSampler(val_data)

# # dataLoader for validation set
# val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
def tokenize_function(example):
    return tokenizer(example, truncation=True)

In [ ]:
# a,b=next(iter(train_dataloader))
# type(a),a 

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
accuracy = evaluate.load("accuracy")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

trainer.train()



In [ ]:


# # freeze all the parameters
# for param in bert.parameters():
#     param.requires_grad = False
     


In [ ]:
# class BERT_Arch(nn.Module):

#     def __init__(self, bert):
      
#       super(BERT_Arch, self).__init__()

#       self.bert = bert 
      
#       # dropout layer
#       self.dropout = nn.Dropout(0.1)
      
#       # relu activation function
#       self.relu =  nn.ReLU()

#       # dense layer 1
#       self.fc1 = nn.Linear(768,512)
      
#       # dense layer 2 (Output layer)
#       self.fc2 = nn.Linear(512,2)

#       #softmax activation function
#       self.softmax = nn.LogSoftmax(dim=1)

#     #define the forward pass
#     def forward(self, sent_id, mask):

#       #pass the inputs to the model  
#       _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
#       x = self.fc1(cls_hs)

#       x = self.relu(x)

#       x = self.dropout(x)

#       # output layer
#       x = self.fc2(x)
      
#       # apply softmax activation
#       x = self.softmax(x)

#       return x

In [ ]:
# model = BERT_Arch(bert)

# push the model to GPU
# model = model.to(device)

In [ ]:


# # optimizer from hugging face transformers
# from transformers import AdamW

# # define the optimizer
# optimizer = AdamW(model.parameters(), lr = 1e-3)
     


In [ ]:
# from sklearn.utils.class_weight import compute_class_weight

# #compute the class weights
# class_wts = compute_class_weight('balanced', np.unique(y_train), y_train)

# print(class_wts)
     




# # convert class weights to tensor
# weights= torch.tensor(class_wts,dtype=torch.float)
# weights = weights.to(device)

# # loss function
# cross_entropy  = nn.NLLLoss(weight=weights) 

# # number of training epochs
# epochs = 10
     
